# Create Spotify playlist from list

In [1]:
import requests
import json
import re
import pandas as pd

title = "(?P<title>\w+)"
artist = "(?P<artist>\w+)"
holder = "(?P<word>\w+)"


# Generate track id list
def append_id(text):
    """extract and append track id to song_list"""
    
    track = re.findall(r'"spotify:track:(.*?)"', text)[0]
    if track is not None:
        song_list.append(track)
    return song_list

def spotify_search(term):
    """search for songs on spotify using api and append_id to song_list"""
    
    if isinstance(term, float):
        return False

    # api request to search
    url = "https://api.spotify.com/v1/search?q="+term+"&type=track"
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.get(url, headers=headers)
    parsed_json = json.loads(r.text)
    
    if parsed_json['tracks']['total'] != 0:
        append_id(r.text)
    else:
        # song unavailable
        return False
    return True


# Generate spotify list
def spotify_add(term):
    """add song to spotify playlist by track id
    """
    # api request to add
    url = "https://api.spotify.com/v1/playlists/"+playlist_id+"/tracks?uris=spotify%3Atrack%3A"+term
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.post(url, headers=headers)
    return




def define_pattern(pattern):
#     pattern = "%tracknumber% %artist% %%"

    var_names = re.findall(r'%(.*?)%', pattern)
    for var_name in var_names:
        pattern = re.sub("%" +var_name + "%", "(?P<" + var_name + r'>\\w+)', pattern)
        pattern = pattern.replace(r'(?P<>\w+)', r'(\w+)')

    print(pattern)
    return pattern


# Text processing methods
def remove_num(text):
    text = re.sub("(\d+)", "", text)
    return text

def remove_sym(text):
    text = re.sub(r'(\W)\1+', r'\1', text)
    text = text.replace(".", " ").replace("，", "").replace("–", "").replace("*", "").replace(
        "-", " ").replace("·", " ").replace("’", " ").replace("/", " ").replace("：", " ").replace(
        "\u3000", "")
    text = text.replace("	", " ").replace("\t", " ")
    text = text.replace("(", " ").replace("（", " ").replace(")", "").replace("）", "").replace("—", " ")
    
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(' +', ' ', text)

    return text

def remove_parensym(text):
    text = text.replace("<", " ").replace(">", "")
    text = text.replace("《", " ").replace("》", " ")
    text = text.replace("[", "").replace("]", "").replace("「", "").replace("」", "")
    return text

def remove_lines(text):
    """remove empty lines"""
    lines = [re.sub(r"ft.*$", "", line.strip()) + "" for line in text.split('\n') if line.strip() != '']
    return lines

def remove_paren(text):
    """remove words in parenthesis"""
    text = re.sub(r"\((.*?)\)", "", text)
    text = re.sub(r"（(.*?)）", "", text)
    return text


# Text processing
def split_lines(line, pos1=0, pos2=-1, sym=" "):
    songs_to_search = line.split(sym)[pos1] + " " + line.split(sym)[pos2]
    return songs_to_search

def skip_lines(lines):
    new_lines = []
    for i in range(len(lines)):
        if i % 2 == 1:
            new_lines.append(lines[i])
    return new_lines

def extract_text(lines, pattern):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
        m = songs_to_search[i]
        
        try:
            m = re.match(pattern, m)
            songs_to_search[i] = ' '.join([m[k] for k in m.groupdict()])
        except:
            # the hardcoded way
            print("\n======hardcode======")
            print(songs_to_search[i])
            split_lines(songs_to_search[i])
            print(songs_to_search[i])
    return songs_to_search


# Hard code to split lines
def split_text(lines, pos1=0, pos2=-1, sym=" "):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
#         print(songs_to_search[i])
#         print(split_lines(songs_to_search[i], pos1, pos2, sym)[0])
        
        
        songs_to_search[i] = split_lines(songs_to_search[i], pos1, pos2, sym)
            
    return songs_to_search


# Web data processing with pandas
def extract_table(link):
    website = pd.read_html(link)
    song_list = []

    all_songs = website[17]
    all_songs

    all_songs.rename(columns=cus_columns, inplace=True)

    if one:
        songs_to_search = all_songs[('十大中文金曲得獎名單（按頒獎次序排序）',  'title')]

    if two:
        all_songs['list1'] = all_songs["title1"] + ' ' +  all_songs["artist1"]
        all_songs['list2'] = all_songs["title2"] + ' ' +  all_songs["artist2"]

        songs_to_search = all_songs["list1"]
        songs_to_search = songs_to_search.append(all_songs["list2"])
        # songs_to_search.drop(index=25, inplace=True)
        
    return songs_to_search


# Extract multiple songs according to artists
def extract_composer(lines, artist_split="：", title_split="，"):
    songs_to_search = []
    
    for line in lines:
#         print(line)
        artist = line.split(artist_split)[0]#[:3]
#         print(artist)
        title_str =  line.replace(artist, "")

        if len(title_split) == 1:
            titles = title_str.split(title_split)
        else:
            pattern = title_split[0] + '(.*?)' + title_split[1]
            titles = re.findall(pattern, title_str)
        print(titles)
            
        for title in titles:
            print(title)
            songs_to_search.append(artist + " " + title)
                
    return songs_to_search

In [2]:
string = """
挨你一枪	NEKO Highway
温柔	五月天 ft. 孙燕姿
真面目	苏慧伦
Better Fly	Bii 毕书尽"""
# text = re.sub(r"ft.*$", "", text)

print(remove_paren(string))


挨你一枪	NEKO Highway
温柔	五月天 ft. 孙燕姿
真面目	苏慧伦
Better Fly	Bii 毕书尽


In [4]:
# pattern = "(\w+) " +title+ "--" +artist+ ""
string = text.splitlines()[0]
print("String: " + string)
# pattern = title+ " " +artist  # + " " +holder

m = re.match(pattern, string)
m.groupdict()
[m[k] for k in m.groupdict()]
string = ' '.join([m[k] for k in m.groupdict()])
print(string)
# print(m.groupdict().values())

String: 


NameError: name 'pattern' is not defined

#### Request for API token.

https://developer.spotify.com/console/post-playlist-tracks/?playlist_id=&position=&uris=

In [16]:
text = """

1.  瘋狂世界／五月天
2.  軍在前哨／鄧麗君
3.  處處留情／周慧敏
4.  認真／張惠妹
5.  害怕／孫燕姿
6.  傻瓜／溫嵐
7.  唱出希望／曾沛慈
8.  終極三國／曾沛慈
9. 看我七十二變／蔡依林
10. 喔耶／盧廣仲
11. 幽默感／范曉萱
12. 一種念頭／王若琳
13. I Won't Last a Day Without You／木匠兄妹
14. 再見螢火蟲／王菲
15. 謝謝儂／陳奕迅
16.旅行的意義／陳綺貞

愛很簡單／陶喆




梦一埸（那英）
天黑黑（孙燕姿）
傻瓜（温岚）
离家出走（丁当）
如果没有你（莫文蔚）
新不了情（万芳）
love(hebe)
Silent all these years （tori amos）
小情歌 （苏打绿）
小丸子的心事 （何韵诗）
下一站天后 （twins）
鱼（陈绮贞）
千年之恋（feat DCW）（信乐团）
逆光 （孙燕姿）

. Dream (Priscilla Ahn)
. 让我想一想 (陈绮贞)
. 玩乐 (方大同)
. 唱歌给你听 (阿牛)
. 香格里拉 (魏如萱)
. 姐，你睡了吗 (徐若瑄)
7. 力量 (罗志祥)
8. 给未来的自己 (梁静茹)
9. 爱情限时批 (万芳 伍佰)

. 够爱
. 好不好(五月天)
. 隐形的翅膀(张韶涵)
. 可不可以不勇敢(范玮琪)
. honey(王心凌)
. 你是我的花朵(伍佰)
. 谢谢(陈奕迅)
爱你一万年 (伍佰) 

　



    .浮夸(陈奕迅)
    .开到荼靡(王菲)
    .A love theme, Olivia、
    有你的快乐(王若琳)
    .青葱(何韵诗)
    .Adia(Sarah McLachlan)
    .想自由(林宥嘉)
    7.那些年 (胡夏、
    好的事情 严爵)

    8.就像白痴一样(卢广仲)
    9.可爱奇怪(曾沛慈)
    .青空(何韵诗)
    .於是长大了以后(谢和弦)
    .人生海海(五月天)
    .萤火虫(曾沛慈)
    .一路上有你(张学友)
    .再见勾勾(卢广仲)

    . 别在我睡著的时候打电话给我(卢广仲)
    . 离开地球表面(五月天)
    . 转转转转运歌(强辩)
    . The SHOW(Lenka)
    . 小路乱撞(曾沛慈)
    . 喜欢你现在的样子(黄韵玲)
    7. 相信(孙燕姿)
    8. 纯真+傻瓜+我的歌声里(五月天、温岚、曲婉婷)
    9. 还是会(韦礼安)
    .我(蔡依林)
    .下一个天亮(郭静)
    .够爱(曾沛慈)
    .终极三国(曾沛慈)
    .你们是我的星光(星光二班)

    7.一个人想著一个人(曾沛慈)
    8.最重要的小事(五月天)

    -----------------------------------------------------------
    . 别在我睡著的时候打电话给我(卢广仲)
    . 离开地球表面(五月天)
    . 转转转转运歌(强辩)
    . The SHOW(Lenka)
    . 小路乱撞(曾沛慈)
    . 喜欢你现在的样子(黄韵玲)
    7. 相信(孙燕姿)
    8. 纯真+数不尽的星空+我的歌声里(五月天、曾沛慈、曲婉婷)
    9. 还是会(韦礼安)
    .嘉宾时间-我想我需要时间(SpeXial)
    .嘉宾时间-Super Style(SpeXial)
    .我(蔡依林)
    .下一个天亮(郭静)
    .够爱(曾沛慈)
    .终极三国(曾沛慈)
    .你们是我的星光(星光二班)

    7.一个人想著一个人(曾沛慈)
    8.最重要的小事(五月天)
    9.傻瓜(温岚)
    .清唱-唱出希望(曾沛慈)
"""

playlist_id = '20NNe0TPUE77zfwWXMxKeq'
token = """
BQC_IbqJZXG-lOuKC9YFnyWeem1jDnUpkL-9him_FnipSjYNpcU0bt_1CpQzQZ7tMFo42StD-DAH843OWhwkr-rFyts7uCasY6JOmn5poJ7crN-7mL4NdaVzxDhXsWijE0iW3oToTYgboA3aEwO9hzF7qKWKYzRcSsgjwvBIRsMdSUVgz7_CT39ysqDI8NnccXuvdg
"""
token = token.strip()


# cus_columns = {"歌曲": "title", "主唱": "artist"}
# cus_columns = {"第一季金曲": "title1", "第二季金曲": "title2", "主唱": "artist1", "主唱.1": "artist2"}
link = None

pattern = "%title% %artist%"

if text:
    pattern = define_pattern(pattern)
    
#     text = remove_paren(text)
    text = text.replace("麽", "麼").replace("王蔻之", "Ivana").replace("许哲佩", "peggy").replace("陈珊妮", "sandee")
    text = text.replace("曲", "").replace("唱", "").replace("词", "")
    text = remove_sym(text)
    text = remove_num(text)
    text = remove_parensym(text)
    lines = remove_lines(text)
#     lines = skip_lines(lines)
    
    """1. remove num"""
    songs_to_search = lines
    
    """2. extract based on pattern"""
#     songs_to_search = extract_text(lines, pattern)

    """3. extract composer"""
#     songs_to_search = extract_composer(lines, artist_split="《", title_split="《》")

    """4. split text manually"""
#     songs_to_search = split_text(lines, 0, 1, " ")
elif link:
    songs_to_search = web_process(link)

(?P<title>\w+) (?P<artist>\w+)


In [17]:
# print(songs_to_search[:5])
print("\n".join([k for k in songs_to_search[:10]]))

瘋狂世界五月天
軍在前哨鄧麗君
處處留情周慧敏
認真張惠妹
害怕孫燕姿
傻瓜溫嵐
出希望曾沛慈
終極三國曾沛慈
看我七十二變蔡依林
喔耶盧廣仲


In [18]:
na_list = []
song_list = []

for i in songs_to_search:
    if not spotify_search(i):
        print("                        ", end="")
        na_list.append(i)
    print(i + " spotify")
    

# create playlist and add
for i in song_list:
    spotify_add(i)
    
print("\n缺：" + ", ".join([na for na in na_list]))

瘋狂世界五月天 spotify
                        軍在前哨鄧麗君 spotify
                        處處留情周慧敏 spotify
認真張惠妹 spotify
害怕孫燕姿 spotify
傻瓜溫嵐 spotify
                        出希望曾沛慈 spotify
終極三國曾沛慈 spotify
                        看我七十二變蔡依林 spotify
                        喔耶盧廣仲 spotify
                        幽默感范曉萱 spotify
一種念頭王若琳 spotify
                        I Wont Last a Day Without You木匠兄妹 spotify
                        再見螢火蟲王菲 spotify
謝謝儂陳奕迅 spotify
                        旅行的意義陳綺貞 spotify
愛很簡單陶喆 spotify
                        梦一埸 那英 spotify
天黑黑 孙燕姿 spotify
傻瓜 温岚 spotify
离家出走 丁当 spotify
如果没有你 莫文蔚 spotify
新不了情 万芳 spotify
love hebe spotify
Silent all these years tori amos spotify
小情歌 苏打绿 spotify
小丸子的心事 何韵诗 spotify
下一站天后 twins spotify
鱼 陈绮贞 spotify
                        千年之恋 feat DCW 信乐团 spotify
逆光 孙燕姿 spotify
Dream Priscilla Ahn spotify
让我想一想 陈绮贞 spotify
玩乐 方大同 spotify
歌给你听 阿牛 spotify
香格里拉 魏如萱 spotify
姐你睡了吗 徐若瑄 spotify
力量 罗志祥 spotify
给未来的自己 梁静茹 spotify
爱情限时批 万芳 伍佰 spotify
够爱 spotify
好不好 五月天

In [17]:
len(songs_to_search)

2000

### Custom 

In [119]:
text = """1987.8 黄莺莺《雪在烧》专辑  《你在我灵魂的边缘》 作曲：陈秀男
1988.9 姜育恒《跟往事干杯》专辑   制作：陈秀男《今夜星光灿烂》，《每一个辗转不眠的夜》作曲：陈秀男
1988.9 星星‧月亮‧太阳《我有自己的路走》专辑  制作：陈秀男《爱情电话》，《心愿》作曲：陈秀男
1988.10 叶欢《记得我们有约》专辑 制作：陈秀男《多说一些说甜一些》，《不眠的夜》作曲：陈秀男
"""

text = re.sub("《(.*?)》专辑", "", text)
text = remove_num(text)
text = remove_sym(text)

lines = remove_line(text)
lines = [line for line in lines if "《" in line]

songs_to_search = extract_composer(lines, artist_split="《", title_split=r"《(.*?)》")

In [47]:
text = """李宗盛：
《生命中的精灵》 《我终于失去了你》《漂洋过海来看你》《领悟》《爱的代价》（梁咏琪和张艾嘉都有唱过）
《寂寞的恋人》（莫文蔚有唱） 《凡人歌》《山丘》 《鬼迷心窍》 《我是真的爱你》 《给自己的歌》《难以抗拒》（李度）《十二楼》（理性与感性作品音乐会现场）
《铿锵玫瑰》（理性与感性作品音乐会现场）《因为寂寞》李宗盛&张艾嘉live
（每首都好听啊，卤煮已经成为老李的脑残粉了）


邓丽君：
《漫步人生路 》《南海姑娘》（王菲有翻唱过，电影&燕尾蝶&里面女主也唱过。我个人喜欢电影版的，很带感）《初恋的地方》《再见我的爱人》《初次尝到寂寞》"""

text = text.replace("：\n", ":")
text = text.replace("：", ":")
text = text.replace("\n《", "《")
lines = remove_line(text)
lines = [line for line in lines if "《" in line]

songs_to_search = extract_composer(lines, artist_split=":", title_split=r"《(.*?)》")

In [123]:
text = """一點點愉快，要用一生來感慨。（選自同恩「一個人遺憾」）

別問我生活累不累，靈魂的事，身體無法解決。（選自曾慶瑜「我背叛了愛」）

如果在你的懷裡甜蜜的死掉，拜託把憔悴的照片都燒掉，最美的一刻要你永遠記得我的好強。（選自徐懷鈺「如果在你的懷裡滿足的死掉」）

你放了太多愛，在我眼裡眉間，可以不用眺望天邊，也能喜悅。（選自蔡淳佳「眼裡眉間」）"""

text = remove_num(text)
text = remove_sym(text)

lines = remove_line(text)

songs_to_search = re.findall(r"（選自(.*?)）", text)

In [55]:
text = """
1. 一剪梅 <费玉清>2. 千千阙歌 <陈慧娴>3. 九百九十九朵玫瑰 <邰正宵>4. 青藏高原 <李娜>5. 上海滩 <叶丽仪>6. 真的好想你 <周冰倩>7. 路边的野花不要采 <邓丽君>8. 铁血丹心 <罗文 甄妮>9. 潇洒走一回 <叶倩文>10. 昨夜星辰 <龙飘飘>11. 最浪漫的事 <赵咏华>12. 梅花三弄 <姜育恒>13. 亲密爱人 <梅艳芳>14. 涛声依旧 <毛宁>15. 万水千山总是情 <汪明荃>16. 酒干倘卖无 <苏芮>17. 把悲伤留给自己 <陈升>18. 舞女泪 <韩宝仪>19. 挪威的森林 <伍佰>20. 独角戏 <许茹芸>21. 千纸鹤 <邰正宵>22. 橄榄树 <齐豫>23. 心雨 <杨钰莹 毛宁>24. 水手 <郑智化>25. 爱江山更爱美人 <李丽芬>26. 星语心愿 <张柏芝>27. 新鸳鸯蝴蝶梦 <黄安>28. 窗外 <李琛>29. 沧海一声笑 <许冠杰>30. 大海 <张雨生>31. 萍聚 <李翊君>32. 用心良苦 <张宇>33. 晚秋 <黄凯芹>34. 伤心太平洋 <任贤齐>35. 小城故事 <邓丽君>36. 偏偏喜欢你 <陈百强>37. 明天你是否依然爱我 <童安格>38. 感恩的心 <欧阳菲菲>39. 女人花 <梅艳芳>40. 单身情歌 <林志炫>41. 阳光总在风雨后 <许美静>42. 雨蝶 <李翊君>43. 外婆的澎湖湾 <潘安邦>44. 风中有朵雨做的云 <孟庭苇>45. 梁祝 <吴奇隆>46. 同桌的你 <老狼>47. 我只在乎你 <邓丽君>48. 懂你 <满文军>49. 再回首 <姜育恒>50. 爱我的人和我爱的人 <裘海正>51. 一千个伤心的理由 <张学友>52. 甜蜜蜜 <邓丽君>53. 特别的爱给特别的你 <伍思凯>54. 千年等一回 <高胜美>55. 朋友别哭 <吕方>56. 祈祷 <王杰 王韵婵>57. 恰似你的温柔 <邓丽君>58. 有多少爱可以重来 <迪克牛仔>59. 追风少年 <吴奇隆>60. 霸王别姬 <屠洪刚>61. 站在高岗上 <张惠妹>62. 大约在冬季 <齐秦>63. 精忠报国 <屠洪刚>64. 很爱很爱你 <刘若英>65. 梦醒时分 <陈淑桦>66. 海浪 <黄品源>67. 盛夏的果实 <莫文蔚>68. 爱你在心口难开 <凤飞飞>69. 但愿人长久 <王菲>70. 敢问路在何方 <西游记>71. 敢问路在何方 <蒋大为>72. 光辉岁月 <Beyond>73. 你究竟有几个好妹妹 <孟庭苇>74. 我被青春撞了一下腰 <张真>75. 过火 <张信哲>76. 爱你十分泪七分 <裘海正>77. 月亮惹的祸 <张宇>78. 相思风雨中 <汤宝如 张学友>79. 别问我是谁 <王馨平>80. 水中花 <谭咏麟>81. 花心 <周华健>82. 孤枕难眠 <周华健>83. 吻别 <张学友>84. 爱你一万年 <刘德华>85. 你的柔情我永远不懂 <陈琳>86. 童年 <罗大佑>87. 男儿当自强 <林子祥>88. 山不转水转 <那英>89. 再回到从前 <张镐哲>90. 恋曲1990 <罗大佑>91. 忘情水 <刘德华>92. 糊涂的爱 <王志文>93. 弯弯的月亮 <刘欢>94. 月亮代表我的心 <邓丽君>95. 宝贝，对不起 <草蜢>96. 其实你不懂我的心 <童安格>97. 爱在深秋 <谭咏麟>98. 爱要怎么说出口 <赵传>99. 不是我不小心 <张镐哲>100. 不装饰你的梦 <蔡国权>101. 被遗忘的时光 <蔡琴>102. 如果还有明天 <刘伟仁>103. 错过你错过爱 <张信哲>104. 不可以不想你 <钟镇涛>105. 大会堂演奏厅 <李克勤>106. 爱就一个字 <张信哲>107. 我恨我痴心 <刘德华>108. 爱之初体验 <张震岳>109. 爱的进行式 <陈淑桦>110. 最近比较烦 <周华健 李宗盛>111. 让世界充满爱 <郭峰>112. 当爱已成往事 <林忆莲 李宗盛>113. 绝口不提爱你 <郑中基>114. 我来自北京 <黎明>115. 轻轻的告诉你 <杨钰莹>116. 谢谢你的爱 <刘德华>117. 怪你过份美丽 <张国荣>118. 卡拉永远OK <谭咏麟>119. 恋曲1980 <罗大佑>120. 改变1995 <黄舒骏>121. 分手的那一夜 <张洪量>122. 星星知我心 <蔡幸娟>123. 一颗不变心 <张学友>124. 东京爱情故事 <小田和正>125. 最真的期待 <赵咏华>126. 孤身走我路 <梅艳芳>127. 美丽的花蝴蝶 <张洪量>128. 你是我的男人 <林忆莲>129. 我想我是海 <黄磊>130. 摆渡人的歌 <周华健>131. 我想有个家 <潘美辰>132. 风中的承诺 <李翊君>133. 想你的时候 <千百惠>134. 一路上有你 <张学友>135. 没有你的日子 <小刚>136. 我们的故事 <赵咏华>137. 最爱上海滩 <刘德华>138. 其实不想走 <周华健>139. 寡妇村传奇 <周华健>140. 千愁记旧情 <罗文>141. 跟往事干杯 <姜育恒>142. 七天七世纪 <李天华>143. 世界第一等 <伍佰>144. 难得有情人 <关淑怡>145. 逝去的诺言 <陈慧娴>146. 摘星的晚上 <李国祥>147. 亲爱的小孩 <苏芮>148. 青苹果乐园 <小虎队>149. 对你爱不完 <郭富城>150. 光阴的故事 <罗大佑>151. 如果云知道 <许茹芸>152. 伤感的恋人 <黄凯芹>153. 好大一棵树 <孙悦>154. 轻抚你的脸 <张学友>155. 有一点动心 <张信哲 刘嘉玲>156. 让一切随风 <钟镇涛>157. 耶利亚女郎 <童安格>158. 说，你爱我 <陈淑桦>159. 我的心太乱 <小刚>160. 真心换绝情 <吴宗宪>161. 真我的风采 <刘德华>162. 忘情森巴舞 <草蜢>163. 只记今朝笑 <吕珊>164. 至少还有你 <林忆莲>165. 我的中国心 <张明敏>166. 永远不回头 <张雨生>167. 捕风的汉子 <谭咏麟>168. 城里的月光 <许美静>169. 何日君再来 <费玉清>170. 外面的世界 <齐秦>171. 似是故人来 <梅艳芳>172. 带我去月球 <张雨生>173. 聪明糊涂心 <陈淑桦>174. 春去春又回 <陈淑桦>175. 一千零一夜 <邰正宵>176. 迟来的春天 <谭咏麟>177. 床前明月光 <梅艳芳>178. 别怕我伤心 <张信哲>179. 真心真意过一生 <叶倩文>180. 穿过你的黑发的我的手 <罗大佑>181. 不让我的眼泪陪我过夜 <齐秦>182. 每次都想呼喊你的名字 <李恕权>183. 输了你赢了世界又如何 <优客李林>184. 我是不是该安静的走开 <郭富城>185. 为什么你背着我爱别人 <许志安>186. 在我生命中的每一天 <陈淑桦 成龙>187. 我是真的付出我的爱 <周华健>188. 穿过你的黑发我的手 <罗大佑>189. 爱上一个不回家的人 <林忆莲>190. 你的眼睛背叛你的心 <郑中基>191. 天下没有不散的筵席 <郑钧>192. 我很丑可是我很温柔 <赵传>193. 为什么受伤的总是我 <林志颖>194. 我是不是你最疼爱的人 <潘越云>195. 我是个需要很多爱的人 <姜育恒>196. 一天一点爱恋 <梁朝伟>197. 一生不可自决 <陈百强>198. 雨中的恋人们 <黄凯芹>199. 月朦胧鸟朦胧 <凤飞飞>200. 祝你一路顺风 <吴奇隆>201. 真的还是假的 <孟庭苇>202. 星光依旧灿烂 <小虎队>203. 最爱的人伤我最深 <张惠妹 张雨生>204. 我该用什么样的心来对你 <黄品源>205. 请不要在别人的肩上哭泣 <赵传>206. 不是每个恋曲都有美好回忆 <林志颖>207. 有没有一首歌会让你想起我 <小刚>208. 是不是这样的夜晚你才会这样的想起我 <吴宗宪>209. 冬季到台北来看雨 <孟庭苇>210. 对面的女孩看过来 <任贤齐>211. 用我一辈子去忘记 <郑智化>212. 原来你什么都不要 <迪克牛仔>213. 一天到晚游泳的鱼 <张雨生>214. 新长征路上的摇滚 <崔健>215. 我这个你不爱的人 <迪克牛仔>216. 无情的雨无情的你 <齐秦>217. 只要你过得比我好 <钟镇涛>218. Made In China <艾敬>219. 虹 <齐秦>220. 说，你爱我 <陈淑桦>221. 对不起,我爱你 <黎明>222. Amani <Beyond>223. Hello <王杰>224. Don t Break My Heart <黑豹>225. 我所爱的让我流泪 <赵传>226. 我是一棵秋天的树 <张雨生>227. 你快乐所以我快乐 <王菲>228. 你怎么舍得我难过 <黄品源>229. 你看你看月亮的脸 <孟庭苇>230. 没有情人的情人节 <孟庭苇>231. 寂寞让我如此美丽 <陈明>232. 可以勇敢可以温柔 <林慧萍>233. 你知道我在等你吗 <张洪量>234. 女人何苦为难女人 <辛晓琪>235. 我和春天有个约会 <刘雅丽>236. 我可以抱你吗爱人 <张惠妹>237. 睡在我上铺的兄弟 <老狼>238. 十九岁的最后一天 <伊能静>239. 认真的女人最美丽 <高慧君>240. 爱你痛到不知痛 <张学友>241. 白天不懂夜的黑 <那英>242. 只想一生跟你走 <张学友>243. 皇后大道东 <罗大佑>244. 最珍惜仍然是你 <梁朝伟>245. 爱一个人好难 <苏永康>246. 宝贝,对不起 <草蜢>247. 爱一回伤一回 <吕方>248. 一天一天等下去 <吴奇隆>249. 一人有一个梦想 <黎瑞恩>250. 我终于失去了你 <赵传>251. 我曾用心爱着你 <潘美辰>252. 想和你去吹吹风 <张学友>253. 亚拉伯跳舞女郎 <Beyond>254. 一场游戏一场梦 <王杰>255. 野百合也有春天 <罗大佑>256. 三月里的小雨 <刘文正>257. 生命中的精灵 <李宗盛>258. 为你朝思暮想 <那英>259. 为了爱梦一生 <王杰>260. 我的1997 <艾敬>261. 我的爱对你说 <叶倩文>262. 我是真的爱你 <张信哲>263. 我拿什么爱你 <齐秦>264. 忘了你忘了我 <王杰>265. 天天等天天问 <叶欢>266. 十七岁的雨季 <林志颖>267. 失恋阵线联盟 <草蜢>268. 谁的心忘了收 <孙楠>269. 谁的眼泪在飞 <孟庭苇>270. 谁说我不在乎 <高明骏>271. 我是一只小小鸟 <赵传>272. 我会像你一样傻 <黎明>273. 明明白白我的心 <陈淑桦>274. 每天爱你多一些 <张学友>275. 那个傻瓜爱过你 <赵传>276. 难以抗拒你容颜 <张信哲>277. 牵挂你的人是我 <高林生>278. 你在他乡还好吗 <李进>279. 今夜你会不会来 <黎明>280. 对不起，我爱你 <黎明>281. 不必在乎我是谁 <林忆莲>282. 白衣飘飘的年代 <叶蓓>283. 春风吻上我的脸 <凤飞飞>284. 等你等到我心痛 <张学友>285. 冬天里的一把火 <费翔>286. 情凭谁来定错对 <谭咏麟>287. 让我欢喜让我忧 <周华健>288. 我爱花香不爱花 <黎明>289. 少年壮志不言愁 <刘欢>290. 我的未来不是梦 <张雨生>291. 我的心没有回程 <姜育恒>292. 我独自在风雨中 <高明骏>293. 我的眼里只有你 <井冈山>294. 如果这都不算爱 <张学友>295. 如果可以再见你 <黎明>296. 让我一次爱个够 <庚澄庆>297. 让我忘记你的脸 <张信哲>298. 人生何处不相逢 <陈慧娴>299. 人在旅途洒泪时 <关正杰>300. 容易受伤的女人 <王菲>301. 相见不如怀念 <那英>302. 星光灿烂 <罗中旭>303. 胭脂扣 <梅艳芳>304. 雪在烧 <黄莺莺>305. 夜来香 <邓丽君>306. 夜上海 <周璇>307. 萤火虫 <伊能静>308. 夜太黑 <林忆莲>309. 心太软 <任贤齐>310. 笑红尘 <陈淑桦>311. 我期待 <张雨生>312. 未央歌 <黄舒骏>313. 我是猫 <伊能静>314. 我应该 <张学友>315. 喜欢你 <Beyond>316. 我愿意 <王菲>317. 正是爱 <陈慧娴>318. 追梦人 <罗大佑>319. 每一步 <徐小凤>320. 垄上行 <张明敏>321. 柠檬树 <苏慧伦>322. 小路 <邓丽君>323. 下沙 <游鸿明>324. 小芳 <李春波>325. 郎归晚 <汪明荃>326. 兰花草 <刘文正>327. 好男人 <张镐哲>328. 自由歌 <张雨生>329. 坏女孩 <梅艳芳>330. 灰姑娘 <郑钧>331. 来生缘 <刘德华>332. 靠近我 <田震>333. 未了情 <郭富城>334. 忘记他 <邓丽君>335. 不了情 <徐小凤>336. 不后悔 <张学友>337. 长城谣 <张明敏>338. 出塞曲 <蔡琴>339. 大中国 <高枫>340. 大国民 <郑智化>341. 玻璃心 <齐秦>342. 白桦林 <朴树>343. 婚纱背后 <徐小凤>344. 回头太难 <张学友>345. 寂寞的眼 <周华健>346. 加州阳光 <张清芳>347. 爱相随 <周华健>348. 爱情鸟 <林依伦>349. 当年情 <张国荣>350. 蝶恋花 <黄凯芹>351. 深深深 <李克勤>352. 含羞草 <雷安娜>353. 失乐园 <陈淑桦>354. 石头记 <达明一派>355. 忘不了 <童安格>356. 四季歌 <达明一派>357. 海上花 <甄妮>358. 归来吧 <陈慧娴>359. 恭喜你 <太极乐队>360. 凡人歌 <李宗盛>361. 走四方 <韩磊>362. 七匹狼 <齐秦>363. 情已逝 <张学友>364. 屋顶 <吴宗宪>365. 味道 <辛晓琪>366. 阴天 <莫文蔚>367. 野花 <田震>368. 不走 <杜德伟>369. 冰雨 <刘德华>370. 渴望 <毛阿敏>371. 安妮 <王杰>372. 鸭子 <苏慧伦>373. 选择 <林子祥 叶倩文>374. 长城 <Beyond>375. 初恋 <红孩儿>376. 彩虹 <羽泉>377. 阿莲 <戴军>378. 信仰 <张信哲>379. 笑脸 <谢东>380. 九妹 <黄鹤翔>381. 姐妹 <张惠妹>382. 值得 <迪克牛仔>383. 直觉 <张信哲>384. 纸船 <许冠杰>385. 祝福 <张学友>386. 虹 <齐秦>387. 最美 <羽泉>388. 执着 <田震>389. 征服 <那英>390. 傻瓜 <苏慧伦>391. 伤痕 <林忆莲>392. 认错 <优客李林>393. 囚鸟 <张宇>394. 掌心 <无印良品>395. 葬月 <张学友>396. 大地 <Beyond>397. 叮咚 <徐怀钰>398. 棋子 <王菲>399. 牵手 <苏芮>400. 飘雪 <陈慧娴>401. 朋友 <周华健>402. 农民 <Beyond>403. 朋友 <臧天朔>404. 倾心 <黄凯芹>405. 情人 <Beyond>406. 天意 <刘德华>407. 偷心 <张学友>408. 天涯 <任贤齐>409. 随缘 <温兆伦>410. 情网 <张学友>411. 说谎 <温兆伦>412. 流星 <王菲>413. 流年 <王菲>414. 红豆 <王菲>415. 红日 <李克勤>416. 附属 <李度>417. 奉献 <苏芮>418. 读你 <费翔>419. 奋斗 <甄妮>420. 后来 <刘若英>421. 灰色 <林忆莲>422. 蓝雨 <张学友>423. 领悟 <辛晓琪>424. 哭砂 <高胜美>425. 姐姐 <张楚>426. 回家 <王杰>427. 回到拉萨 <郑钧>428. 顺流逆流 <徐小凤>429. 真心英雄 <周华健>430. 真的汉子 <林子祥>431. 之乎者也 <罗大佑>432. 知心爱人 <付笛生 任静>433. 钟爱一生 <杜德伟>434. 执迷不悔 <王菲>435. 真的爱你 <Beyond>436. 曾经心疼 <叶倩文>437. 因为寂寞 <张艾嘉>438. 驿动的心 <姜育恒>439. 勇敢一点 <赵传>440. 雨一直下 <张宇>441. 在水一方 <邓丽君>442. 再见理想 <Beyond>443. 祝你平安 <孙悦>444. 最爱是谁 <林子祥>445. 为你钟情 <张国荣>446. 为爱痴狂 <刘若英>447. 我不想说 <杨钰莹>448. 无地自容 <黑豹>449. 风再起时 <张国荣>450. 无心睡眠 <张国荣>451. 偷偷爱你 <梁朝伟>452. 天天想你 <张雨生>453. 说声珍重 <王杰>454. 最真的梦 <周华健>455. 似水流年 <梅艳芳>456. 太想爱你 <张信哲>457. 天若有情 <高胜美>458. 天长地久 <周启生>459. 一言难尽 <张宇>460. 一笑而过 <那英>461. 拒绝再玩 <张国荣>462. 镜花水月 <梅艳芳>463. 口是心非 <张雨生>464. 快乐老家 <陈明>465. 纤夫的爱 <尹相杰 于文华>466. 雾里看花 <那英>467. 男孩不许哭 <钟镇涛>468. 那天再重聚 <欧瑞强>469. 两忘烟水里 <关正杰>470. 今宵多珍重 <陈百强>471. 绿岛小夜曲 <蔡琴>472. 漫步人生路 <邓丽君>473. 命运是你家 <Beyond>474. 明天会更好 <群星合唱>475. 现在以后 <童安格>476. 乡间小路 <刘文正>477. 一封家书 <李春波>478. 亚洲雄风 <韦唯 刘欢>479. 一生何求 <陈百强>480. 一生一次 <谭咏麟>481. 一无所有 <崔健>482. 一世情缘 <童安格>483. 星星点灯 <郑智化>484. 狼 <齐秦>485. 小村之恋 <邓丽君>486. 想哭就哭 <姜育恒>487. 笑看风云 <郑少秋>488. 心的方向 <周华健>489. 心如刀割 <张学友>490. 刀剑如梦 <周华健>491. 得意的笑 <李丽芬>492. 冷酷到底 <羽泉>493. 浪子心声 <许冠杰>494. 流光飞舞 <陈淑桦>495. 梦回唐朝 <唐朝>496. 南海姑娘 <王菲>497. 那些花儿 <朴树>498. 浪人情歌 <伍佰>499. 来来回回 <张学友>500. 不再犹豫 <Beyond>501. 不见不散 <孙楠>502. 沉默是金 <许冠杰>503. 春光乍泄 <黄耀明>504. 爱的代价 <张艾嘉>505. 爱不爱我 <零点>506. 南屏晚钟 <徐小凤>507. 难舍难分 <谭咏麟>508. 伤心的我 <邝美云>509. 让爱继续 <谭咏麟>510. 伤心小站 <蔡琴>511. 少女心事 <张国荣>512. 北方的狼 <齐秦>513. 深情相拥 <张国荣 辛晓琪>514. 情深无怨 <谭咏麟>515. 青春舞曲 <罗大佑>516. 你的眼神 <蔡琴>517. 难以抗拒 <李度>518. 你的样子 <罗大佑>519. 你快回来 <孙楠>520. 千言万语 <邓丽君>521. 年轻时代 <郑智化>522. 北京一夜 <陈升>523. 报告班长 <庾澄庆>524. 鬼迷心窍 <李宗盛>525. 故乡的云 <费翔>526. 滚滚红尘 <陈淑桦>527. 哈萨雅琪 <周传雄>528. 海阔天空 <Beyond>529. 海角天涯 <刘锡明>530. 共同度过 <张国荣>531. 干杯朋友 <田震>532. 堕落天使 <郑智化>533. 东方之珠 <罗大佑>534. 饿狼传说 <张学友>535. 非常夏日 <王菲>536. 风雨无阻 <周华健>537. 风继续吹 <张国荣>538. 好好爱我 <张蔷>539. 黑色午夜 <张国荣>540. 爱人同志 <罗大佑>541. 爱情故事 <辛晓琪>542. 爱如潮水 <张信哲>543. 爱与哀愁 <童安格>544. 半梦半醒 <谭咏麟>545. 把根留住 <童安格>546. 爱的奉献 <翁倩玉>547. 黄色月亮 <苏慧伦>548. 护花使者 <李克勤>549. 蝴蝶飞呀 <小虎队>550. 花房姑娘 <崔健>551. 花季未了 <刘若英>552. 还是朋友 <张雨生>553. 讲不出再见 <谭咏麟>
"""

text = text.replace("<", "").replace("   500首经典老歌。  ", "").replace("</p><p>", "")
text = text.replace("叶倩文", "sally")

lines = text.split(">")
lines = [remove_num(line).strip() for line in lines]
lines = [remove_sym(line) for line in lines]
songs_to_search = lines

print("\n".join([line for line in lines]))

  一剪梅 费玉清
  千千阙歌 陈慧娴
  九百九十九朵玫瑰 邰正宵
  青藏高原 李娜
  上海滩 叶丽仪
  真的好想你 周冰倩
  路边的野花不要采 邓丽君
  铁血丹心 罗文 甄妮
  潇洒走一回 sally
  昨夜星辰 龙飘飘
  最浪漫的事 赵咏华
  梅花三弄 姜育恒
  亲密爱人 梅艳芳
  涛声依旧 毛宁
  万水千山总是情 汪明荃
  酒干倘卖无 苏芮
  把悲伤留给自己 陈升
  舞女泪 韩宝仪
  挪威的森林 伍佰
  独角戏 许茹芸
  千纸鹤 邰正宵
  橄榄树 齐豫
  心雨 杨钰莹 毛宁
  水手 郑智化
  爱江山更爱美人 李丽芬
  星语心愿 张柏芝
  新鸳鸯蝴蝶梦 黄安
  窗外 李琛
  沧海一声笑 许冠杰
  大海 张雨生
  萍聚 李翊君
  用心良苦 张宇
  晚秋 黄凯芹
  伤心太平洋 任贤齐
  小城故事 邓丽君
  偏偏喜欢你 陈百强
  明天你是否依然爱我 童安格
  感恩的心 欧阳菲菲
  女人花 梅艳芳
  单身情歌 林志炫
  阳光总在风雨后 许美静
  雨蝶 李翊君
  外婆的澎湖湾 潘安邦
  风中有朵雨做的云 孟庭苇
  梁祝 吴奇隆
  同桌的你 老狼
  我只在乎你 邓丽君
  懂你 满文军
  再回首 姜育恒
  爱我的人和我爱的人 裘海正
  一千个伤心的理由 张学友
  甜蜜蜜 邓丽君
  特别的爱给特别的你 伍思凯
  千年等一回 高胜美
  朋友别哭 吕方
  祈祷 王杰 王韵婵
  恰似你的温柔 邓丽君
  有多少爱可以重来 迪克牛仔
  追风少年 吴奇隆
  霸王别姬 屠洪刚
  站在高岗上 张惠妹
  大约在冬季 齐秦
  精忠报国 屠洪刚
  很爱很爱你 刘若英
  梦醒时分 陈淑桦
  海浪 黄品源
  盛夏的果实 莫文蔚
  爱你在心口难开 凤飞飞
  但愿人长久 王菲
  敢问路在何方 西游记
  敢问路在何方 蒋大为
  光辉岁月 Beyond
  你究竟有几个好妹妹 孟庭苇
  我被青春撞了一下腰 张真
  过火 张信哲
  爱你十分泪七分 裘海正
  月亮惹的祸 张宇
  相思风雨中 汤宝如 张学友
  别问我是谁 王馨平
  水中花 谭咏麟
  花心 周华健
  孤枕难眠 周华健
  吻别 张学友
  爱你一万年 刘德华
  你的柔情我永远不懂 陈琳
  童

In [ ]:
text= """## Linfair

#### 兩情相樂
01. 原來的世界 - 范曉萱+王力宏
02. 將愛實現 - 邰正宵+王菲
03. 清者自清 - 蘇永康+范曉萱
04. 仰慕 - 周慧敏+陳立強
05. 相約1999 - 范曉萱+王英州
06. Sometimes When We Touch - 邰正宵+周慧敏
07. 都市戀愛故事 - CD Voice
08. 山頂黑狗兄 - 林隆璇+張中立
09. 忠於愛情 - 林隆璇+馬玉芬
10. 走在大街的女子 - 陳耀川+周慧敏
11. 知己 - 邰正宵+周慧敏
12. 流言 - 周慧敏+林隆璇 

"""

print(re.sub("\d+:\d+", "",text.replace("–", "").replace("-", "").replace("*", "").replace("\t", " ").replace("  ", " ").replace(". ", ".")))